In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
from statistics import NormalDist
import pickle
import networkx as nx

### Thesis Plots

#### Website count

In [ ]:
df = pd.read_csv('input/thesis_WebsitesInfo.csv')

In [ ]:
fig = px.bar(data_frame=df, x='URL', y='Count', 
             text_auto='.2s', text='Percentage')

texts = [f'{round(value, 2)}%' for value in df['Percentage'].values]

#fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(texttemplate = texts, textposition = 'outside')
fig.update_layout(yaxis_range=[0, 60000])
fig.show()

In [ ]:
fig.write_html('output/WebsitesAnalysis.html',
                full_html=False,
                include_plotlyjs='cdn')

#### Description Test Results

In [ ]:
df = pd.read_csv('input/thesis_TestPrediction.csv')
df['bin'] = df.index.values / df.index.values[-1]

In [ ]:
def create_KDE(df, column, bw=None, num=501):
    x = np.linspace(0, 1, num=num)
    kernel = gaussian_kde(df[column], bw_method=bw) 
    y = kernel.pdf(x)
    
    return x, y

In [ ]:
x, y = create_KDE(df, 'Probability_1', num=101)

In [ ]:
correct_all = []
incorrect_all = []
for x1, x2 in zip(x, x[1:]):
    df_subset = df[(df['bin'].between(x1, x2))].groupby('Correct').count()
    try:
        total = df_subset.iloc[1, 0] + df_subset.iloc[0, 0]
        correct = df_subset.iloc[0, 0] / total
    except:
        correct = correct_all[-1]
    incorrect = 1 - correct
    
    correct_all.append(correct)
    incorrect_all.append(incorrect)

In [ ]:
correct_all.append(correct_all[-1])
incorrect_all.append(incorrect_all[-1])

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

x, y = create_KDE(df[df['Correct'] == 'Correct'], 'Probability_1', num=101)

fig.add_trace(go.Scatter(
    x=x, y=correct_all,
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='green'),
    stackgroup='one',
    groupnorm='percent'
))

x, y = create_KDE(df[df['Correct'] == 'Incorrect'], 'Probability_1', num=101)

fig.add_trace(go.Scatter(
    x=x, y=incorrect_all,
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='red'),
    stackgroup='one',
    showlegend=True,
))

fig.update_layout(
    showlegend=False,
    #xaxis_type='category',
    yaxis=dict(
        type='linear',
        range=[1, 100],
        ticksuffix='%'))

x, y = create_KDE(df, 'Probability_1')

fig.add_trace(
    go.Scatter(x=x, y=y, name="Description Density",
              line=dict(width=1.5, color='rgb(131, 90, 241)'),
               showlegend=True,
              ),
    secondary_y=True,
)

fig.layout.yaxis2.update(showticklabels=False, showgrid= False)

# Set x-axis title
fig.update_xaxes(title_text="Prediction Values")

# Set y-axes titles
fig.update_yaxes(title_text="Correct/Incorrect", secondary_y=False)
fig.update_yaxes(title_text="Data Distribution", secondary_y=True)

In [ ]:
fig.write_html('output/Classifier_testAccuracy.html',
                full_html=False,
                include_plotlyjs='cdn')

### Taxon Distance

In [2]:
df = pd.read_csv('input/TaxonDistance.csv')

In [3]:
df = df.set_index(['Unnamed: 0', 'Unnamed: 1'])\
       .rename_axis(['taxon', 'type'])

In [4]:
taxons = df.index.get_level_values('taxon').unique()

In [5]:
taxons

Index(['kingdom', 'phylum', 'class', 'order', 'family'], dtype='object', name='taxon')

In [ ]:
#df.xs('phylum', level='taxon')

In [ ]:
%%capture
for taxon in taxons[0:4]:
    
    X = df.xs(taxon, level='taxon').round(3)
    X[X < 0.05] = np.NaN


    x_labels = X.columns
    y_labels = X.index

    fig = px.imshow(X,
                    labels=dict(x="Classifier", y="Taxon", color="Distance"),
                    x=x_labels,
                    y=y_labels,
                    text_auto=True, aspect="auto",
                    color_continuous_scale='viridis_r',
                   )
    fig.update_xaxes(side="top")
    #fig.update_layout(height=800)
    #fig.show()
    
    fig.write_html(f'output/distance_{taxon}.html',
                full_html=False,
                include_plotlyjs='cdn')

#### Xarray

In [ ]:
distance_dict = pickle.load(open('input/TaxonDistanceDict.pkl', 'rb'))

In [ ]:
taxons = list(distance_dict.keys())

In [ ]:
taxons

In [ ]:
xarr = distance_dict['class'].to_xarray().to_array()

In [ ]:
xarr = xarr.rename({'variable': 'classifier'})

In [ ]:
xarr[0: 0, 0, 0]

In [ ]:
fig = px.imshow(xarr[0, :, :],
                text_auto=True, aspect="auto",
                color_continuous_scale='plasma_r',
               )
fig.update_xaxes(side="top")
fig.update_layout(height=800)
fig.show()

In [ ]:
df = px.data.gapminder()
px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

### Paulien

In [ ]:
df = pd.read_excel('input/AllData_allyears.xlsx')

In [ ]:
df['date_str'] = df['Date'].apply(lambda x: str(x))
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

In [ ]:
gardens = df['Sampling.area'].unique()
losses = ['TP_log', 'TN_log', 'PO4_log', 'NO3_log']

df_std = df.groupby([df.index, 'Sampling.area']).std()
df_mean = df.groupby([df.index, 'Sampling.area']).mean()
df_mean.reset_index(inplace=True)
df_std.reset_index(inplace=True)

fig = go.Figure().set_subplots(4, 1, vertical_spacing=0.05,
                              shared_xaxes=True,
                              shared_yaxes=True,
                              subplot_titles=gardens)

for idx, garden in enumerate(gardens):
    for loss in losses:
    mean_subset = df_mean[df_mean['Sampling.area'] == garden]
    std_subset = df_std[df_std['Sampling.area'] == garden]
    
    loss = [
    go.Scatter(
        name='TN log',
        x=mean_subset['Date'],
        y=mean_subset['TN_log'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
    ),
    go.Scatter(
        name='Upper Bound',
        x=std_subset['Date'],
        y=mean_subset['TN_log']+std_subset['TN_log'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False,
    ),
    go.Scatter(
        name='Lower Bound',
        x=std_subset['Date'],
        y=mean_subset['TN_log']-std_subset['TN_log'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
    ]
    
    fig.add_trace(loss[0], row=idx + 1, col=1)
    fig.add_trace(loss[1], row=idx + 1, col=1)
    fig.add_trace(loss[2], row=idx + 1, col=1)
    
fig.update_layout(
    hovermode="x",
    height=800,
)

#fig.for_each_yaxis(lambda y: y.update(title = ''))
fig.add_annotation(x=-0.02,y=0.5,
                   text="Loss (log)", textangle=-90,
                    xref="paper", yref="paper",
                  font=dict(size=20,))


fig.show()    
# fig.write_html(f'output/VoorPaulienTest.html',
#         full_html=False,
#         include_plotlyjs='cdn')

In [ ]:
mean_subset = df_mean[df_mean['Sampling.area'] == 'PAR']
std_subset = df_std[df_std['Sampling.area'] == 'PAR']

colors = [ '50,  92,  168', 
          '168, 135,   50', 
          '168,  58,   50', 
           '50, 168,  150',]
losses = ['TP_log', 'TN_log', 'PO4_log', 'NO3_log']


fig = go.Figure()

for loss, color in zip(losses, colors):
    conf = [
    go.Scatter(
        name=loss.split("_")[0],
        x=mean_subset['Date'],
        y=mean_subset[loss],
        mode='lines',
        line=dict(color=f'rgb({color})'),
        showlegend=True,
        legendgroup=loss,
    ),
    go.Scatter(
        name='Upper Bound',
        x=std_subset['Date'],
        y=mean_subset[loss]+std_subset[loss],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0.5),
        showlegend=False,
        hovertemplate=None,
        hoverinfo='skip', 
        legendgroup=loss,

    ),
    go.Scatter(
        name='Lower Bound',
        x=std_subset['Date'],
        y=mean_subset[loss]-std_subset[loss],
        marker=dict(color="#444"),
        line=dict(width=0.5),
        mode='lines',
        fillcolor=f'rgba({color}, 0.3)',
        fill='tonexty',
        showlegend=False,
        hoverinfo='skip', 
        legendgroup=loss,   
    )
    ]

    fig.add_trace(conf[0], )#row=idx + 1, col=1)
    fig.add_trace(conf[1], )#row=idx + 1, col=1)
    fig.add_trace(conf[2], )#row=idx + 1, col=1)
    
fig.update_layout(
    hovermode="x",
    #hovermode='x unified',
    #height=800,
    #title="Losses",
    xaxis_title="",
    yaxis_title="Losses (log)",
    legend_title="Loss Type",
    font=dict(
        #family="Courier New, monospace",
        size=10,
        color="black"
    )
)


#fig.update_traces(mode="markers+lines")
fig.show()